# Refinement
This notebook will show how to perform a simple form of structure refinement

## Generate artificial data

We will here generate some scattering data, to refine against

In [12]:
from iotbx import pdb
from pathlib import Path

# Use a tyrosine in a P1 box
structure_file = Path().resolve() / "data" / "tyrosine.pdb"

# Load structure
xrs = pdb.input(file_name=str(structure_file)).xray_structure_simple()
xrs.scattering_type_registry(table="xray")

# Generate Fobs
f_obs = xrs.structure_factors(d_min=0.8).f_calc().as_amplitude_array()

# Shake the structure a little, to have something to refine against
xrs.shake_sites_in_place(rms_difference=0.1)

True

## Refinement loop
Here we show a simple gradient descent refinement with gradients from pydiscamb.

We use the least squares target function defined as:
$$ LSQ(x, y) = \frac{\Sigma_i\left(x_i - \left|y_i\right|\right)^2}{\Sigma_i x_i^2}, $$

where $x$ is $F_\text{obs}$ and $y$ is $F_\text{calc}$.
The gradient with respect to $F_\text{calc}$ is
$$ \frac{\partial }{\partial y_i}LSQ(x, y) = \frac{-2 \cdot \left(x_i - \left|y_i\right|\right) }{\Sigma_i x_i^2}\frac{y_i}{|y_i|}, $$
which is passed to DiSCaMB to compute the gradients of the target function with respect to atomic parameters.

In [ ]:
import pydiscamb
from cctbx.array_family import flex

# Set up wrapper
wrapper = pydiscamb.DiscambWrapper(xrs)
wrapper.set_indices(f_obs.indices())

# Set up target functor
def least_squares_d_target_d_f_calc(f_obs, f_calc) -> list[complex]:
    o = f_obs.data()
    c = f_calc.data()
    a = flex.abs(c)
    return -2 * (o - a) / a * c / flex.sum(o*o)

## Refinement loop
for cycle in range(20):
    # Compute gradients
    f_calc = f_obs.array(data = wrapper.f_calc())
    print(f"{cycle = :2}, R: {f_obs.r1_factor(f_calc) :5.1%}")
    d_target_d_f_calc = least_squares_d_target_d_f_calc(f_obs, f_calc)
    grads = wrapper.d_target_d_params(list(d_target_d_f_calc))

    # Update structure
    for grad, scatterer in zip(grads, xrs.scatterers()):
        scatterer.site = (
            scatterer.site[0] - 0.1 * grad.site_derivatives[0],
            scatterer.site[1] - 0.1 * grad.site_derivatives[1],
            scatterer.site[2] - 0.1 * grad.site_derivatives[2],
        )
    wrapper.update_structure(xrs)


cycle =  0, R: 11.4%
cycle =  1, R:  6.0%
cycle =  2, R:  3.9%
cycle =  3, R:  3.0%
cycle =  4, R:  2.5%
cycle =  5, R:  2.3%
cycle =  6, R:  2.2%
cycle =  7, R:  2.1%
cycle =  8, R:  2.0%
cycle =  9, R:  1.9%
cycle = 10, R:  1.9%
cycle = 11, R:  1.8%
cycle = 12, R:  1.8%
cycle = 13, R:  1.7%
cycle = 14, R:  1.7%
cycle = 15, R:  1.7%
cycle = 16, R:  1.7%
cycle = 17, R:  1.6%
cycle = 18, R:  1.6%
cycle = 19, R:  1.6%
